In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [49]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [50]:
#create new DataFrame to retrieve all the rows where the total_votes count is equal to or greater than 20
plus20votes = vine_df.filter("total_votes>=20")
plus20votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [51]:
# Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpfulvotes = plus20votes.filter("(helpful_votes/total_votes)>= 0.5")
helpfulvotes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [53]:
# retrieves all the rows where a review was written as part of the Vine program (paid)
paidreviews_df = helpfulvotes.filter(helpfulvotes["vine"] == "Y")
paidreviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|
|R1W3KCDHM6BMZT|          4|           37|         38|   Y|                N|
|R3KDMPS96IJ10R|          2|           15|         21|   Y|                N|
|R1XWEI58A7VKLB|          5|          123|        128|   Y|     

In [54]:
# retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaidreviews_df = helpfulvotes.filter(helpfulvotes["vine"] == "N")
unpaidreviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [55]:
# Total number of paid reviews
from pyspark.sql.functions import count
paidreviews_total_Number = paidreviews_df.select("review_id").count()
print("Paid total number of reviews: ", paidreviews_total_Number)

Paid total number of reviews:  107


In [56]:
# Number of stars breakdown for paid reviews
paidreviews_5stars = paidreviews_df.filter(paidreviews_df["star_rating"] == "5").count()
paidreviews_4stars = paidreviews_df.filter(paidreviews_df["star_rating"] == "4").count()
paidreviews_3stars = paidreviews_df.filter(paidreviews_df["star_rating"] == "3").count()
paidreviews_2stars = paidreviews_df.filter(paidreviews_df["star_rating"] == "2").count()
paidreviews_1stars = paidreviews_df.filter(paidreviews_df["star_rating"] == "1").count()
print("5 Star:", paidreviews_5stars)
print("4 Star:", paidreviews_4stars)
print("3 Star:", paidreviews_3stars)
print("2 Star:", paidreviews_2stars)
print("1 Star:", paidreviews_1stars)

5 Star: 56
4 Star: 32
3 Star: 16
2 Star: 2
1 Star: 1


In [57]:
# Total number of unpaid reviews
unpaidreviews_total_Number = unpaidreviews_df.select("review_id").count()
print("Unpaid total number of reviews: ", unpaidreviews_total_Number)

Unpaid total number of reviews:  39869


In [58]:
# Number of stars breakdown for unpaid reviews
unpaidreviews_5stars = unpaidreviews_df.filter(unpaidreviews_df["star_rating"] == "5").count()
unpaidreviews_4stars = unpaidreviews_df.filter(unpaidreviews_df["star_rating"] == "4").count()
unpaidreviews_3stars = unpaidreviews_df.filter(unpaidreviews_df["star_rating"] == "3").count()
unpaidreviews_2stars = unpaidreviews_df.filter(unpaidreviews_df["star_rating"] == "2").count()
unpaidreviews_1stars = unpaidreviews_df.filter(unpaidreviews_df["star_rating"] == "1").count()
print("5 Star:", unpaidreviews_5stars)
print("4 Star:", unpaidreviews_4stars)
print("3 Star:", unpaidreviews_3stars)
print("2 Star:", unpaidreviews_2stars)
print("1 Star:", unpaidreviews_1stars)

5 Star: 21005
4 Star: 7384
3 Star: 3744
2 Star: 2412
1 Star: 5324


In [59]:
summary_df1 = spark.createDataFrame([("5", paidreviews_5stars, unpaidreviews_5stars),
                                   ("4", paidreviews_4stars, unpaidreviews_4stars),
                                   ("3", paidreviews_3stars, unpaidreviews_3stars),
                                   ("2", paidreviews_2stars, unpaidreviews_2stars),
                                   ("1", paidreviews_1stars, unpaidreviews_1stars),
                                   ("Total", paidreviews_total_Number, unpaidreviews_total_Number)], ["Stars", "Number of PAID reviews", "Number of UNPAID reviews"])
summary_df1.show()

+-----+----------------------+------------------------+
|Stars|Number of PAID reviews|Number of UNPAID reviews|
+-----+----------------------+------------------------+
|    5|                    56|                   21005|
|    4|                    32|                    7384|
|    3|                    16|                    3744|
|    2|                     2|                    2412|
|    1|                     1|                    5324|
|Total|                   107|                   39869|
+-----+----------------------+------------------------+



In [60]:
summary_df2 = spark.createDataFrame([("5", (paidreviews_5stars/paidreviews_total_Number)*100, (unpaidreviews_5stars/unpaidreviews_total_Number)*100),
                                   ("4", (paidreviews_4stars/paidreviews_total_Number)*100, (unpaidreviews_4stars/unpaidreviews_total_Number)*100),
                                   ("3", (paidreviews_3stars/paidreviews_total_Number)*100, (unpaidreviews_3stars/unpaidreviews_total_Number)*100),
                                   ("2", (paidreviews_2stars/paidreviews_total_Number)*100, (unpaidreviews_2stars/unpaidreviews_total_Number)*100),
                                   ("1", (paidreviews_1stars/paidreviews_total_Number)*100, (unpaidreviews_1stars/unpaidreviews_total_Number)*100),
                                   ("Total", (paidreviews_total_Number/paidreviews_total_Number)*100, (unpaidreviews_total_Number/unpaidreviews_total_Number)*100)], ["Stars", "% of PAID reviews", "% of UNPAID reviews"])
summary_df2.show()

+-----+------------------+-------------------+
|Stars| % of PAID reviews|% of UNPAID reviews|
+-----+------------------+-------------------+
|    5|52.336448598130836|  52.68504351751988|
|    4|29.906542056074763| 18.520655145601847|
|    3|14.953271028037381|  9.390754721713613|
|    2|1.8691588785046727|  6.049813138027039|
|    1|0.9345794392523363| 13.353733477137625|
|Total|             100.0|              100.0|
+-----+------------------+-------------------+

